# Homework 3 
## Romina Doz
### Algorithmic Design - 18/05/2021

#### 1) Implement the binary heap-based version of the Dijkstra’s algorithm.

To implement the Dijikstra's algorithm, we can use the adjacency list representation for the graph and the binary heap class that was implemented during the lessons. The problem of implementing the algorithm using a binary heap as the queue data structure is the fact that we don't know the position of each node inside the heap. So, when we want to update the distance of a node in the heap, we have to find it. The complexity of updating a node become $O(n * log(n))$

In [1]:
from binheap import binheap
import sys

In [2]:
class edge():
    def __init__(self, dest, w, is_short = False, is_rev = False):
        self.destination = dest
        self.weight = w
        self.is_shortcut = is_short
        self.is_reversed = is_rev
        self.is_removed = False
        
    def __repr__(self) -> str:
        return f' EDGE: Destination: {self.destination.val} ; weight: {self.weight} ; is shortcut: {self.is_shortcut} ; is reversed: {self.is_reversed}'

In [3]:
class graphnode():
    def __init__(self, val, d = -1, inu = False, ind = False):
        self.val = val
        self.d = d
        self.pred = None
        self.edges = list()
        self.in_u = inu
        self.in_d = ind
    
    def add_edge(self, destination, weight, short = False, rev = False):
        self.edges.append(edge(destination, weight, short, rev))
        
    def __repr__(self) -> str:
        return f'NODE: Value: {self.val} ; distance from source: {self.d} ; predecessor: {self.pred.val if self.pred else None}'

In [4]:
def total_order_nodes(a:(int,graphnode), b:(int,graphnode))->bool:
    return a[0] <= b[0]

In [5]:
def find_in_heap(value,Q):
    count = 0
    for element in Q._A:
        if element[1] == value:
            if(count <= len(Q)):
                return count
            else:
                raise Exception("The heap does not contain the node") 
        count = count + 1        
    return None

In [6]:
class graph():
    def __init__(self, size = 0):
        self._size = size
        self.nodes = list()
        
    def add_node(self, node):
        self.nodes.append(node)
        self._size = self._size
                
    def print_nodes_distance(self):
        print('Graph:')
        for node in self.nodes:
            print(node)
            
    def print_edges(self):
        print('Nodes and edges:')
        for node in self.nodes:
            print("---------------------------")
            print(node)
            for e in node.edges:
                print(e)
            
    def init_sssp(self):
        for node in self.nodes:
            node.d = sys.maxsize
            node.pred = None
    
    @staticmethod
    def relax(Q: binheap, u:graphnode, v:graphnode, w:int)->None:
        if u.d + w < v.d:
            index = find_in_heap(v, Q)
            v.d = u.d + w
            Q.decrease_key(index,(v.d,v))
            v.pred = u

    def dijkstra(self, s):
        self.init_sssp()
        s.d = 0
        
        lst_nodes = [(node.d, node) for node in self.nodes]
        
        Q = binheap(lst_nodes, total_order = total_order_nodes)
        
        while not Q.is_empty():
            u = Q.remove_minimum()
            
            for edge in u[1].edges:
                self.relax(Q, u[1], edge.destination, edge.weight)                        

We can test the algorithm replicating the same example of the course's slides

In [7]:
node1 = graphnode(1)
node2 = graphnode(2)
node3 = graphnode(3)
node4 = graphnode(4)
node5 = graphnode(5)
node6 = graphnode(6)

node1.add_edge(node2,1)
node1.add_edge(node3,5)
node2.add_edge(node6,15)
node3.add_edge(node4,2)
node4.add_edge(node5,1)
node5.add_edge(node6,3)

graph1 = graph()
graph1.add_node(node1)
graph1.add_node(node2)
graph1.add_node(node3)
graph1.add_node(node4)
graph1.add_node(node5)
graph1.add_node(node6)

graph1.print_nodes_distance()
graph1.dijkstra(node1)
print("---------------------------")
print("After Dijikstra's algorithm")
print("---------------------------")
graph1.print_nodes_distance()



Graph:
NODE: Value: 1 ; distance from source: -1 ; predecessor: None
NODE: Value: 2 ; distance from source: -1 ; predecessor: None
NODE: Value: 3 ; distance from source: -1 ; predecessor: None
NODE: Value: 4 ; distance from source: -1 ; predecessor: None
NODE: Value: 5 ; distance from source: -1 ; predecessor: None
NODE: Value: 6 ; distance from source: -1 ; predecessor: None
---------------------------
After Dijikstra's algorithm
---------------------------
Graph:
NODE: Value: 1 ; distance from source: 0 ; predecessor: None
NODE: Value: 2 ; distance from source: 1 ; predecessor: 1
NODE: Value: 3 ; distance from source: 5 ; predecessor: 1
NODE: Value: 4 ; distance from source: 7 ; predecessor: 3
NODE: Value: 5 ; distance from source: 8 ; predecessor: 4
NODE: Value: 6 ; distance from source: 11 ; predecessor: 5


#### 2) Consider the contraction hierarchies presented during the course. Assume to deal with graphs that can be fully represented in the memory of your computer. Implement:

#### (a) An algorithm to add the shortcuts to a graph;

We consider that the importance of the node is given by its value.

In [8]:
def hierarchy(a:graphnode, b:graphnode)->bool:
    return a.val <= b.val
                    
def has_better_path(n_from:graphnode, n_to:graphnode, value) -> bool:
    for edge in n_from.edges:
        if ((edge.destination == n_to and edge.weight <= value) or n_from == n_to ):
            return True
        if(edge.destination == n_to and edge.weight > value and edge.is_shortcut):
            n_from.edges.remove(edge)
    return False

def add_shortcuts_to_node(Q:binheap, node_remove:graphnode) -> None:
    for i in range(len(Q)):
        node = Q._A[i]
        for edge in node.edges:
            if edge.destination == node_remove:
                for e in node_remove.edges:
                    if not has_better_path(node, e.destination, edge.weight + e.weight):
                        node.add_edge(e.destination, edge.weight + e.weight, True)

def add_shortcuts_to_graph(g:graph) -> None:    
    Q = binheap(g.nodes, total_order = hierarchy)
    while not Q.is_empty():
        u = Q.remove_minimum()
        add_shortcuts_to_node(Q, u)

We can test the algorithm replicating the same example of the course's slides

In [9]:
node1 = graphnode(1)
node2 = graphnode(2)
node3 = graphnode(3)
node4 = graphnode(4)
node5 = graphnode(5)
node6 = graphnode(6)
node7 = graphnode(7)
node8 = graphnode(8)

node1.add_edge(node5,1)
node1.add_edge(node6,1)
node2.add_edge(node3,2)
node3.add_edge(node2,1)
node3.add_edge(node4,3)
node4.add_edge(node7,1)
node4.add_edge(node8,3)
node5.add_edge(node1,3)
node5.add_edge(node6,1)
node6.add_edge(node8,1)
node7.add_edge(node8,1)
node8.add_edge(node1,1)
node8.add_edge(node7,1)

graph2 = graph()
graph2.add_node(node1)
graph2.add_node(node2)
graph2.add_node(node3)
graph2.add_node(node4)
graph2.add_node(node5)
graph2.add_node(node6)
graph2.add_node(node7)
graph2.add_node(node8)

graph2.print_edges()
add_shortcuts_to_graph(graph2)
print("***************************")
print("WITH SHORTCUTS")
print("***************************")
graph2.print_edges()



Nodes and edges:
---------------------------
NODE: Value: 1 ; distance from source: -1 ; predecessor: None
 EDGE: Destination: 5 ; weight: 1 ; is shortcut: False ; is reversed: False
 EDGE: Destination: 6 ; weight: 1 ; is shortcut: False ; is reversed: False
---------------------------
NODE: Value: 2 ; distance from source: -1 ; predecessor: None
 EDGE: Destination: 3 ; weight: 2 ; is shortcut: False ; is reversed: False
---------------------------
NODE: Value: 3 ; distance from source: -1 ; predecessor: None
 EDGE: Destination: 2 ; weight: 1 ; is shortcut: False ; is reversed: False
 EDGE: Destination: 4 ; weight: 3 ; is shortcut: False ; is reversed: False
---------------------------
NODE: Value: 4 ; distance from source: -1 ; predecessor: None
 EDGE: Destination: 7 ; weight: 1 ; is shortcut: False ; is reversed: False
 EDGE: Destination: 8 ; weight: 3 ; is shortcut: False ; is reversed: False
---------------------------
NODE: Value: 5 ; distance from source: -1 ; predecessor: None
 

#### (b) A bidirectional version of Dijkstra algorithm that can operate on the graphs decorated by the algorithm at Point 2a.

In [15]:
#After running the algorithm remove backward edges
def restore(g:graph):
    for node in g.nodes:
        node.in_u = False
        node.in_v = False
        node.edges = [x for x in node.edges if not x.is_reversed]
        for e in node.edges:
            e.is_removed = False

#Before running the algorithm create backward edges
def create_backward_edges(n:graphnode):
    for e in n.edges:
        if n.val > e.destination.val:
            e.is_removed = True
            e.destination.add_edge(n, e.weight, False, True)
    

def bi_dijkstra(g:graph, source:graphnode, target:graphnode):
    
    print('Running bidirectional Dijkstra:')
    print('-------------------------------')
    
    #BASE CASES
    if source == target:
        print('-------------------------------')
        return 0
    
    for edge in source.edges:
        if edge.destination == target:
            print('-------------------------------')
            return edge.weight
    
    #INITIALIZATION AND CREATION OF HEAPS
    g.init_sssp()
    source.d = 0
    target.d = 0
    source.in_u = True
    target.in_d = True
    lst_nodes_u = list()
    lst_nodes_d = list()
    for node in g.nodes:
        create_backward_edges(node)          
        if node == source:
            lst_nodes_u.append((node.d,node))
            lst_nodes_d.append((sys.maxsize,node))
        elif node == target:
            lst_nodes_d.append((node.d,node))
            lst_nodes_u.append((sys.maxsize,node)) 
        else:
            lst_nodes_u.append((node.d,node))
            lst_nodes_d.append((node.d,node))
    
    Q_U = binheap(lst_nodes_u, total_order = total_order_nodes)
    Q_D = binheap(lst_nodes_d, total_order = total_order_nodes)
    
    total_dist = sys.maxsize
    keep_iterating_on_U = True
    keep_iterating_on_D = True
    
    #ITERATION
    while not Q_U.is_empty() and not Q_D.is_empty():
        if keep_iterating_on_U:
            u = Q_U.remove_minimum()
            print(f'Minimum of Q_U : {u[1].val}')
        if keep_iterating_on_D:
            v = Q_D.remove_minimum()
            print(f'Minimum of Q_D : {v[1].val}')
        
        #Stop condition
        if(u[0] + v[0] >= total_dist):
            restore(g)
            print('-------------------------------')
            return total_dist
       
        #Forward Search
        if keep_iterating_on_U:
            keep_iterating_on_U = False
            for edge in u[1].edges:
                if not edge.is_reversed and not edge.is_removed:
                    keep_iterating_on_U = True
                    edge.destination.in_u = True
                    if edge.destination.in_d and edge.destination.d + u[1].d + edge.weight < total_dist:
                        total_dist = edge.destination.d + u[1].d + edge.weight  
                    g.relax(Q_U, u[1], edge.destination, edge.weight)
                    
        #Backward Search
        if keep_iterating_on_D:
            keep_iterating_on_D = False
            for edge in v[1].edges:
                if edge.is_reversed and not edge.is_removed:
                    keep_iterating_on_D = True
                    edge.destination.in_d = True
                    if edge.destination.in_u and edge.destination.d + v[1].d + edge.weight < total_dist:
                        total_dist = edge.destination.d + v[1].d + edge.weight
                    g.relax(Q_D, v[1], edge.destination, edge.weight)                

We can use the previous graph to calculate the distance between two nodes:

In [20]:
node_from = node2
node_to = node1
distance = bi_dijkstra(graph2, node_from, node_to)
print(f'Total Distance between nodes {node_from.val} and {node_to.val} is equal to {distance}')

print('*******************************')
print('*******************************')

node_from = node4
node_to = node5
distance = bi_dijkstra(graph2, node_from, node_to)
print(f'Total Distance between nodes {node_from.val} and {node_to.val} is equal to {distance}')

Running bidirectional Dijkstra:
-------------------------------
Minimum of Q_U : 2
Minimum of Q_D : 1
Minimum of Q_U : 3
Minimum of Q_D : 8
Minimum of Q_U : 4
Minimum of Q_U : 7
Minimum of Q_U : 5
-------------------------------
Total Distance between nodes 2 and 1 is equal to 8
*******************************
*******************************
Running bidirectional Dijkstra:
-------------------------------
Minimum of Q_U : 4
Minimum of Q_D : 5
Minimum of Q_U : 7
Minimum of Q_D : 8
Minimum of Q_U : 8
-------------------------------
Total Distance between nodes 4 and 5 is equal to 4
